<a href="https://colab.research.google.com/github/JoshuaHarper98/MM908-Industrial-Project/blob/main/Preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import sys
sys.path.append('/content/gdrive/My Drive/Colab Notebooks')

In [ ]:
import numpy as np
import pandas as pd
import math
from tqdm import tqdm
import re
from bs4 import BeautifulSoup

In [ ]:
!pip install textacy
import textacy
from textacy.extract import keyterms as kt
from textacy import preprocessing

     |████████████████████████████████| 200 kB 7.5 MB/s 
     |████████████████████████████████| 2.0 MB 86.3 MB/s 
     |████████████████████████████████| 477 kB 77.4 MB/s 
     |████████████████████████████████| 6.4 MB 56.7 MB/s 
     |████████████████████████████████| 456 kB 91.7 MB/s 
     |████████████████████████████████| 42 kB 2.0 MB/s 
     |████████████████████████████████| 621 kB 75.9 MB/s 
     |████████████████████████████████| 10.1 MB 64.4 MB/s 
  Created wheel for cytoolz: filename=cytoolz-0.11.0-cp37-cp37m-linux_x86_64.whl size=1224739 sha256=fcb475ad19473a0e16be249b04dd5192e132ea95620a74fc5fa858a874271d35
  Stored in directory: /root/.cache/pip/wheels/a1/aa/85/7dc0547cad8c535a1c4e55c638f6e2d3de45401dd634bff0a2
Successfully built cytoolz
  Attempting uninstall: catalogue
    Found existing installation: catalogue 1.0.0
    Uninstalling catalogue-1.0.0:
      Successfully uninstalled catalogue-1.0.0
  Attempting uninstall: srsly
    Found existing installation: srsly 1.0.5

In [ ]:
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
!python -m spacy download en_core_web_sm
nlp = spacy.load('en_core_web_sm')

2021-08-05 14:50:55.780770: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
     |████████████████████████████████| 13.6 MB 7.5 MB/s 
  Attempting uninstall: en-core-web-sm
    Found existing installation: en-core-web-sm 2.2.5
    Uninstalling en-core-web-sm-2.2.5:
      Successfully uninstalled en-core-web-sm-2.2.5
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [ ]:
stopwords = set(STOP_WORDS)

In [ ]:
nlp.pipe_names

['tok2vec', 'tagger', 'parser', 'attribute_ruler', 'lemmatizer', 'ner']

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/Data/classification_data.csv")
df.head()

,Unnamed: 0,POLICYID,Text,Product_Code,Claim,Number_of_Claims
0,1,100218,Set null on Primary Named Policyholder to 'No ...,CPO,No,0
1,2,100235,Set null on Primary Named Policyholder to 'No ...,CPO,No,0
2,3,100288,NONE,CPO,No,0
3,4,100423,NOW BEING DEBITTED ON GW\r\nNO CLAIMS\r\nON CO...,CPO,No,0
4,5,100449,Set null on Primary Named Policyholder to 'No ...,CPO,No,0


In [ ]:
# df = df.sample(frac=0.01, replace=True, random_state=1)

df = df.dropna()
df.head()

,Unnamed: 0,POLICYID,Text,Product_Code,Claim,Number_of_Claims
0,1,100218,Set null on Primary Named Policyholder to 'No ...,CPO,No,0
1,2,100235,Set null on Primary Named Policyholder to 'No ...,CPO,No,0
2,3,100288,NONE,CPO,No,0
3,4,100423,NOW BEING DEBITTED ON GW\r\nNO CLAIMS\r\nON CO...,CPO,No,0
4,5,100449,Set null on Primary Named Policyholder to 'No ...,CPO,No,0


In [ ]:
df['Claim'].value_counts()

No     121530
Yes     30436
Name: Claim, dtype: int64

In [ ]:
df['Text'] = df['Text'].apply(lambda x: BeautifulSoup(x,'lxml').get_text())
df.head()

/usr/local/lib/python3.7/dist-packages/bs4/__init__.py:273: UserWarning: "b'.'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)


,Unnamed: 0,POLICYID,Text,Product_Code,Claim,Number_of_Claims
0,1,100218,Set null on Primary Named Policyholder to 'No ...,CPO,No,0
1,2,100235,Set null on Primary Named Policyholder to 'No ...,CPO,No,0
2,3,100288,NONE,CPO,No,0
3,4,100423,NOW BEING DEBITTED ON GW\r\nNO CLAIMS\r\nON CO...,CPO,No,0
4,5,100449,Set null on Primary Named Policyholder to 'No ...,CPO,No,0


In [ ]:
df['emails'] = df['Text'].apply(lambda x: re.findall(r'([a-zA-Z0-9+._-]+@[a-zA-Z0-9._-]+\.[a-zA-Z0-9_-]+)', x))
df['emails_count'] = df['emails'].apply(lambda x: len(x))
df[df['emails_count']>0].head()

,Unnamed: 0,POLICYID,Text,Product_Code,Claim,Number_of_Claims,emails,emails_count
49,50,107534,R2021 file handling sheet has been uploaded to...,CPO,No,0,[raymillardinsurance@raymillardinsurance.co.uk],1
76,77,192216,U/W issue(s):\r\n\r\n1. Liability Total Techni...,CPO,No,0,[rsouthgate@yahoo.co.uk],1
80,81,192236,U/W issue(s):\r\n\r\n1. Liability Total Techni...,CPO,No,0,[rsouthgate@yahoo.co.uk],1
599,600,65507370,Changed Address Line 1 on Policy Address from ...,CCI,No,0,[WRFMAIL@AVIVA.COM],1
600,601,65509955,----------------------------------------------...,CCI,No,0,[WRFMAIL@AVIVA.COM],1


In [ ]:
df['Text'] = df['Text'].apply(lambda x: textacy.preprocessing.replace.emails(x, repl = ' '))
df[df['emails_count']>0].head()

,Unnamed: 0,POLICYID,Text,Product_Code,Claim,Number_of_Claims,emails,emails_count
49,50,107534,R2021 file handling sheet has been uploaded to...,CPO,No,0,[raymillardinsurance@raymillardinsurance.co.uk],1
76,77,192216,U/W issue(s):\r\n\r\n1. Liability Total Techni...,CPO,No,0,[rsouthgate@yahoo.co.uk],1
80,81,192236,U/W issue(s):\r\n\r\n1. Liability Total Techni...,CPO,No,0,[rsouthgate@yahoo.co.uk],1
599,600,65507370,Changed Address Line 1 on Policy Address from ...,CCI,No,0,[WRFMAIL@AVIVA.COM],1
600,601,65509955,----------------------------------------------...,CCI,No,0,[WRFMAIL@AVIVA.COM],1


In [ ]:
df['urls'] = df['Text'].apply(lambda x: (re.findall(r'(http|ftp|https)://([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:/~+#-]*[\w@?^=%&/~+#-])?', x)))
df['url_flags'] = df['urls'].apply(lambda x: len(x))
df['Text'] = df['Text'].apply(lambda x: re.sub(r'(http|ftp|https)://([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:/~+#-]*[\w@?^=%&/~+#-])?', '', x))
df[df['url_flags']>0].head()

,Unnamed: 0,POLICYID,Text,Product_Code,Claim,Number_of_Claims,emails,emails_count,urls,url_flags
14,15,101458,U/W issue(s):\r\n\r\n1. Liability Total Techni...,CPO,No,0,[],0,"[(https, web.tpa-cit.apps.aws-euw1.avivacloud....",2
35,36,105146,Rationale 2020 \r\n\r\nTrade - Property Owner ...,CPO,No,0,[],0,"[(https, web.tpa-cit.apps.aws-euw1.avivacloud....",1
84,85,192304,"Claim free,\r\nHawkeye all Green (Loc 2 is are...",CPO,No,0,[],0,"[(https, web.tpa-cit.apps.aws-euw1.avivacloud....",1
226,227,197106,No claims as per CIT\r\n\r\n2 commercial premi...,CPO,No,0,[],0,"[(https, web.tpa-cit.apps.aws-euw1.avivacloud....",1
268,269,199218,Strategy / Business\r\n\r\nCommercial property...,CPO,No,0,[],0,"[(https, web.tpa-cit.apps.aws-euw1.avivacloud....",2


In [ ]:
df['Text'] = df['Text'].apply(lambda x: textacy.preprocessing.replace.numbers(x, repl = ' '))

In [ ]:
limit = 0

def cleaner(df):
    "Extract relevant text from DataFrame using a regex"
    # Regex pattern for only alphanumeric, hyphenated text with 2 or more chars
    pattern = re.compile(r"[A-Za-z0-9\-]{2,50}")
    df['clean'] = df['Text'].str.findall(pattern).str.join(' ')
    if limit > 0:
        return df.iloc[:limit, :].copy()
    else:
        return df

In [ ]:
df_preproc = cleaner(df)
print(df.Text.head(5))
print(df_preproc.clean.head(5))

0    Set null on Primary Named Policyholder to 'No ...
1    Set null on Primary Named Policyholder to 'No ...
2                                                 NONE
3    NOW BEING DEBITTED ON GW\r\nNO CLAIMS\r\nON CO...
4    Set null on Primary Named Policyholder to 'No ...
Name: Text, dtype: object
0    Set null on Primary Named Policyholder to No P...
1    Set null on Primary Named Policyholder to No P...
2                                                 NONE
3    NOW BEING DEBITTED ON GW NO CLAIMS ON COVE SIN...
4    Set null on Primary Named Policyholder to No P...
Name: clean, dtype: object


In [ ]:
!pip install contractions

     |████████████████████████████████| 283 kB 9.3 MB/s 
     |████████████████████████████████| 321 kB 58.7 MB/s 
  Created wheel for pyahocorasick: filename=pyahocorasick-1.4.2-cp37-cp37m-linux_x86_64.whl size=85474 sha256=910981f0991b380709fb654ace5db51854bca40c9ac922aba335b9f7583a1bf8
  Stored in directory: /root/.cache/pip/wheels/25/19/a6/8f363d9939162782bb8439d886469756271abc01f76fbd790f
Successfully built pyahocorasick


In [ ]:
import contractions

In [ ]:
df_preproc['clean'] = df_preproc['clean'].apply(lambda x: contractions.fix(x))
df_preproc.head()

,Unnamed: 0,POLICYID,Text,Product_Code,Claim,Number_of_Claims,emails,emails_count,urls,url_flags,clean
0,1,100218,Set null on Primary Named Policyholder to 'No ...,CPO,No,0,[],0,[],0,Set null on Primary Named Policyholder to No P...
1,2,100235,Set null on Primary Named Policyholder to 'No ...,CPO,No,0,[],0,[],0,Set null on Primary Named Policyholder to No P...
2,3,100288,NONE,CPO,No,0,[],0,[],0,NONE
3,4,100423,NOW BEING DEBITTED ON GW\r\nNO CLAIMS\r\nON CO...,CPO,No,0,[],0,[],0,NOW BEING DEBITTED ON GW NO CLAIMS ON COVE SIN...
4,5,100449,Set null on Primary Named Policyholder to 'No ...,CPO,No,0,[],0,[],0,Set null on Primary Named Policyholder to No P...


In [ ]:
def extract_lemmas(doc, **kwargs):
    return [t.lemma_ for t in textacy.extract.words(doc, **kwargs)]

In [ ]:
def extract_entities(doc, include_types=None, sep='_'):

    ents = textacy.extract.entities(doc, 
             include_types=include_types, 
             exclude_types=None, 
             drop_determiners=True, 
             min_freq=1)
    
    return [sep.join([t.lemma_ for t in e])+'/'+e.label_ for e in ents]

In [ ]:
def extract_nlp(doc):
    return {
    'lemmas'          : extract_lemmas(doc, exclude_pos = ['PART', 'PUNCT', 
                                        'DET', 'PRON', 'SYM', 'SPACE'],
                                     filter_stops = False),
    'adjs_verbs'      : extract_lemmas(doc, include_pos = ['ADJ', 'VERB']),
    'nouns'           : extract_lemmas(doc, include_pos = ['NOUN', 'PROPN']),
    'entities'        : extract_entities(doc, ['PERSON', 'ORG', 'GPE', 'LOC'])
    }

In [ ]:
nlp_columns = list(extract_nlp(nlp.make_doc('')).keys())
print(nlp_columns)

['lemmas', 'adjs_verbs', 'nouns', 'entities']


In [ ]:
for col in nlp_columns:
    df_preproc[col] = None

In [ ]:
df_preproc.head()

,Unnamed: 0,POLICYID,Text,Product_Code,Claim,Number_of_Claims,emails,emails_count,urls,url_flags,clean,lemmas,adjs_verbs,nouns,entities
0,1,100218,Set null on Primary Named Policyholder to 'No ...,CPO,No,0,[],0,[],0,Set null on Primary Named Policyholder to No P...,None,None,None,None
1,2,100235,Set null on Primary Named Policyholder to 'No ...,CPO,No,0,[],0,[],0,Set null on Primary Named Policyholder to No P...,None,None,None,None
2,3,100288,NONE,CPO,No,0,[],0,[],0,NONE,None,None,None,None
3,4,100423,NOW BEING DEBITTED ON GW\r\nNO CLAIMS\r\nON CO...,CPO,No,0,[],0,[],0,NOW BEING DEBITTED ON GW NO CLAIMS ON COVE SIN...,None,None,None,None
4,5,100449,Set null on Primary Named Policyholder to 'No ...,CPO,No,0,[],0,[],0,Set null on Primary Named Policyholder to No P...,None,None,None,None


In [ ]:
batch_size = 50
batches = math.ceil(len(df_preproc) / batch_size)
docs = []

for i in tqdm(range(0, len(df), batch_size), total=batches):
    docs = nlp.pipe(df_preproc['clean'][i:i+batch_size])

100%|██████████| 3040/3040 [00:00<00:00, 31738.49it/s]


In [ ]:
batch_size = 50
batches = math.ceil(len(df_preproc) / batch_size)

for i in tqdm(range(0, len(df), batch_size), total=batches):
    docs = nlp.pipe(df_preproc['clean'][i:i+batch_size])
    
    for j, doc in enumerate(docs):
        for col, values in extract_nlp(doc).items():
            df_preproc[col].iloc[i+j] = values

  0%|          | 0/3040 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
100%|██████████| 3040/3040 [1:43:59<00:00,  2.05s/it]


In [ ]:
df_preproc.head()

,Unnamed: 0,POLICYID,Text,Product_Code,Claim,Number_of_Claims,emails,emails_count,urls,url_flags,clean,lemmas,adjs_verbs,nouns,entities
0,1,100218,Set null on Primary Named Policyholder to 'No ...,CPO,No,0,[],0,[],0,Set null on Primary Named Policyholder to No P...,"[set, null, on, Primary, Named, Policyholder, ...",[true],"[set, null, Primary, Named, Policyholder, PAYE...",[]
1,2,100235,Set null on Primary Named Policyholder to 'No ...,CPO,No,0,[],0,[],0,Set null on Primary Named Policyholder to No P...,"[set, null, on, Primary, Named, Policyholder, ...",[true],"[set, null, Primary, Named, Policyholder, PAYE...",[]
2,3,100288,NONE,CPO,No,0,[],0,[],0,NONE,[none],[],[],[]
3,4,100423,NOW BEING DEBITTED ON GW\r\nNO CLAIMS\r\nON CO...,CPO,No,0,[],0,[],0,NOW BEING DEBITTED ON GW NO CLAIMS ON COVE SIN...,"[now, be, debitte, ON, GW, CLAIMS, on, COVE, s...",[debitte],"[GW, CLAIMS, COVE, CHANGES, APPLIED, INC]",[]
4,5,100449,Set null on Primary Named Policyholder to 'No ...,CPO,No,0,[],0,[],0,Set null on Primary Named Policyholder to No P...,"[set, null, on, Primary, Named, Policyholder, ...",[true],"[set, null, Primary, Named, Policyholder, PAYE...",[]


In [ ]:
def lemmatize_pipe(doc):
    lemma_list = [str(tok.lemma_).lower() for tok in doc
                  if tok.is_alpha and tok.text.lower() not in stopwords] 
    return lemma_list

def preprocess_pipe(texts):
    preproc_pipe = []
    for doc in nlp.pipe(texts, batch_size=2000):
        preproc_pipe.append(lemmatize_pipe(doc))
    return preproc_pipe

In [ ]:
from joblib import Parallel, delayed

def chunker(iterable, total_length, chunksize):
    return (iterable[pos: pos + chunksize] for pos in range(0, total_length, chunksize))

def flatten(list_of_lists):
    "Flatten a list of lists to a combined list"
    return [item for sublist in list_of_lists for item in sublist]

def process_chunk(texts):
    preproc_pipe = []
    for doc in nlp.pipe(texts, batch_size=2000):
        preproc_pipe.append(lemmatize_pipe(doc))
    return preproc_pipe

def preprocess_parallel(texts, chunksize=100):
    executor = Parallel(n_jobs=7, backend='multiprocessing', prefer="processes")
    do = delayed(process_chunk)
    tasks = (do(chunk) for chunk in chunker(texts, len(df_preproc), chunksize=chunksize))
    result = executor(tasks)
    return flatten(result)

In [ ]:
%%time
df_preproc['tokens'] = preprocess_parallel(df_preproc['clean'], chunksize=1000)

In [ ]:
df_preproc.head()

In [ ]:
df_preproc.to_pickle("text_cleaned.pkl")